In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
data = pd.read_csv('/content/student_data.csv')
data = pd.get_dummies(data)

In [ ]:
num_features = len(data.columns)
latent_dim = 100  # Example dimension of latent space
num_epochs = 100
batch_size = 32

In [ ]:
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(num_features, activation='sigmoid'))
    return model

In [ ]:
def build_discriminator(num_features):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_dim=num_features))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [ ]:
discriminator = build_discriminator(num_features)
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
generator = build_generator(latent_dim)
gan_input = tf.keras.Input(shape=(latent_dim,))
fake_student = generator(gan_input)
gan_output = discriminator(fake_student)
gan = tf.keras.models.Model(gan_input, gan_output)
discriminator.trainable = False
gan.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
X_train = data.values

In [ ]:
discriminator_losses = []
for epoch in range(num_epochs):
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_student_data = generator.predict(noise)
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_student_data = X_train[idx]
    d_loss_real = discriminator.train_on_batch(real_student_data, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_student_data, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator_losses.append(d_loss)
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))
    print(f"Epoch: {epoch+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

threshold = 0.1

1/1 [==============================] - 0s 78ms/step
Epoch: 1, Discriminator Loss: 0.623881921172142, Generator Loss: 0.6012363433837891
1/1 [==============================] - 0s 17ms/step
Epoch: 2, Discriminator Loss: 0.37799350963905454, Generator Loss: 0.7477388381958008
1/1 [==============================] - 0s 15ms/step
Epoch: 3, Discriminator Loss: 0.30502695264294744, Generator Loss: 0.8845016360282898
1/1 [==============================] - 0s 15ms/step
Epoch: 4, Discriminator Loss: 0.2506387762259692, Generator Loss: 1.0242806673049927
1/1 [==============================] - 0s 17ms/step
Epoch: 5, Discriminator Loss: 0.21092747536022216, Generator Loss: 1.175811529159546
1/1 [==============================] - 0s 16ms/step
Epoch: 6, Discriminator Loss: 0.17401077318936586, Generator Loss: 1.3445982933044434
1/1 [==============================] - 0s 23ms/step
Epoch: 7, Discriminator Loss: 0.14326616236940026, Generator Loss: 1.5263469219207764
1/1 [==============================] - 0s 16ms/step
Epoch: 8, Discriminator Loss: 0.11896649457048625, Generator Loss: 1.7140936851501465
1/1 [==============================] - 0s 16ms/step
Epoch: 9, Discriminator Loss: 0.1002053152769804, Generator Loss: 1.9125033617019653
1/1 [==============================] - 0s 17ms/step
Epoch: 10, Discriminator Loss: 0.07865146896801889, Generator Loss: 2.097902536392212
1/1 [==============================] - 0s 15ms/step
Epoch: 11, Discriminator Loss: 0.06858023791573942, Generator Loss: 2.264525890350342
1/1 [==============================] - 0s 15ms/step
Epoch: 12, Discriminator Loss: 0.059400064405053854, Generator Loss: 2.460902452468872
1/1 [==============================] - 0s 15ms/step
Epoch: 13, Discriminator Loss: 0.052326171193271875, Generator Loss: 2.5929198265075684
1/1 [==============================] - 0s 15ms/step
Epoch: 14, Discriminator Loss: 0.0480084412265569, Generator Loss: 2.6160435676574707
1/1 [==============================] - 0s 15ms/step
Epoch: 15, Discriminator Loss: 0.04277648194693029, Generator Loss: 2.716498613357544
1/1 [==============================] - 0s 15ms/step
Epoch: 16, Discriminator Loss: 0.04477113636676222, Generator Loss: 2.8231496810913086
1/1 [==============================] - 0s 16ms/step
Epoch: 17, Discriminator Loss: 0.04812898358795792, Generator Loss: 2.7353367805480957
1/1 [==============================] - 0s 16ms/step
Epoch: 18, Discriminator Loss: 0.048902221489697695, Generator Loss: 2.6726746559143066
1/1 [==============================] - 0s 16ms/step
Epoch: 19, Discriminator Loss: 0.05273164715617895, Generator Loss: 2.573610782623291
1/1 [==============================] - 0s 21ms/step
Epoch: 20, Discriminator Loss: 0.05618555401451886, Generator Loss: 2.548572540283203
1/1 [==============================] - 0s 17ms/step
Epoch: 21, Discriminator Loss: 0.06638636766001582, Generator Loss: 2.308764696121216
1/1 [==============================] - 0s 16ms/step
Epoch: 22, Discriminator Loss: 0.07778745144605637, Generator Loss: 2.305426597595215
1/1 [==============================] - 0s 15ms/step
Epoch: 23, Discriminator Loss: 0.08074514009058475, Generator Loss: 2.150315999984741
1/1 [==============================] - 0s 16ms/step
Epoch: 24, Discriminator Loss: 0.08781463885679841, Generator Loss: 2.0781378746032715
1/1 [==============================] - 0s 16ms/step
Epoch: 25, Discriminator Loss: 0.08119455527048558, Generator Loss: 2.1698315143585205
1/1 [==============================] - 0s 16ms/step
Epoch: 26, Discriminator Loss: 0.07811469794251025, Generator Loss: 2.205110549926758
1/1 [==============================] - 0s 15ms/step
Epoch: 27, Discriminator Loss: 0.0714961844496429, Generator Loss: 2.232905149459839
1/1 [==============================] - 0s 17ms/step
Epoch: 28, Discriminator Loss: 0.07327522244304419, Generator Loss: 2.2079858779907227
1/1 [==============================] - 0s 15ms/step
Epoch: 29, Discriminator Loss: 0.07841819641180336, Generator Loss: 2.117684841156006
1/1 [==============================] - 0s 16ms/step
Epoch: 30, Discriminator Loss: 0.09519316325895488, Generator Loss: 1.9037699699401855
1/1 [==============================] - 0s 16ms/step
Epoch: 31, Discriminator Loss: 0.10685841155645903, Generator Loss: 1.9119536876678467
1/1 [==============================] - 0s 23ms/step
Epoch: 32, Discriminator Loss: 0.1676519401371479, Generator Loss: 1.5567981004714966
1/1 [==============================] - 0s 16ms/step
Epoch: 33, Discriminator Loss: 0.14461377894986072, Generator Loss: 1.5028477907180786
1/1 [==============================] - 0s 15ms/step
Epoch: 34, Discriminator Loss: 0.14328525392693336, Generator Loss: 1.5727874040603638
1/1 [==============================] - 0s 19ms/step
Epoch: 35, Discriminator Loss: 0.12006446369491641, Generator Loss: 1.8247655630111694
1/1 [==============================] - 0s 15ms/step
Epoch: 36, Discriminator Loss: 0.09065800912048871, Generator Loss: 2.100806713104248
1/1 [==============================] - 0s 15ms/step
Epoch: 37, Discriminator Loss: 0.058778542443178594, Generator Loss: 2.451446294784546
1/1 [==============================] - 0s 18ms/step
Epoch: 38, Discriminator Loss: 0.14741291105747223, Generator Loss: 2.2268967628479004
1/1 [==============================] - 0s 15ms/step
Epoch: 39, Discriminator Loss: 0.0646307051390167, Generator Loss: 2.0562539100646973
1/1 [==============================] - 0s 16ms/step
Epoch: 40, Discriminator Loss: 0.07536807667592463, Generator Loss: 2.0066170692443848
1/1 [==============================] - 0s 18ms/step
Epoch: 41, Discriminator Loss: 0.07644799351775818, Generator Loss: 2.0706005096435547
1/1 [==============================] - 0s 17ms/step
Epoch: 42, Discriminator Loss: 0.06758494675159593, Generator Loss: 2.209115505218506
1/1 [==============================] - 0s 17ms/step
Epoch: 43, Discriminator Loss: 0.0593094453215601, Generator Loss: 2.3162426948547363
1/1 [==============================] - 0s 23ms/step
Epoch: 44, Discriminator Loss: 0.06289426982403142, Generator Loss: 2.3510942459106445
1/1 [==============================] - 0s 16ms/step
Epoch: 45, Discriminator Loss: 0.08316199481487413, Generator Loss: 2.0380606651306152
1/1 [==============================] - 0s 15ms/step
Epoch: 46, Discriminator Loss: 0.11929800361397531, Generator Loss: 1.8437061309814453
1/1 [==============================] - 0s 15ms/step
Epoch: 47, Discriminator Loss: 0.09969729185105547, Generator Loss: 1.9891248941421509
1/1 [==============================] - 0s 15ms/step
Epoch: 48, Discriminator Loss: 0.06749355793388179, Generator Loss: 2.345831871032715
1/1 [==============================] - 0s 15ms/step
Epoch: 49, Discriminator Loss: 0.0441960617958044, Generator Loss: 2.6477911472320557
1/1 [==============================] - 0s 16ms/step
Epoch: 50, Discriminator Loss: 0.03268334404694939, Generator Loss: 2.9105160236358643
1/1 [==============================] - 0s 16ms/step
Epoch: 51, Discriminator Loss: 0.02492551893493765, Generator Loss: 3.1507606506347656
1/1 [==============================] - 0s 15ms/step
Epoch: 52, Discriminator Loss: 0.019623678457431737, Generator Loss: 3.3737239837646484
1/1 [==============================] - 0s 16ms/step
Epoch: 53, Discriminator Loss: 0.01585414593864698, Generator Loss: 3.5737528800964355
1/1 [==============================] - 0s 16ms/step
Epoch: 54, Discriminator Loss: 0.013412490574410185, Generator Loss: 3.755411148071289
1/1 [==============================] - 0s 16ms/step
Epoch: 55, Discriminator Loss: 0.13995973765850067, Generator Loss: 3.730556011199951
1/1 [==============================] - 0s 15ms/step
Epoch: 56, Discriminator Loss: 0.02153980638831854, Generator Loss: 3.6669247150421143
1/1 [==============================] - 0s 16ms/step
Epoch: 57, Discriminator Loss: 0.013257704578328422, Generator Loss: 3.6296920776367188
1/1 [==============================] - 0s 16ms/step
Epoch: 58, Discriminator Loss: 0.0136376563706107, Generator Loss: 3.6145472526550293
1/1 [==============================] - 0s 18ms/step
Epoch: 59, Discriminator Loss: 0.01369759067893916, Generator Loss: 3.618767023086548
1/1 [==============================] - 0s 15ms/step
Epoch: 60, Discriminator Loss: 0.013555457815562546, Generator Loss: 3.63270902633667
1/1 [==============================] - 0s 15ms/step
Epoch: 61, Discriminator Loss: 0.01333882659673733, Generator Loss: 3.6541314125061035
1/1 [==============================] - 0s 16ms/step
Epoch: 62, Discriminator Loss: 0.0130143715068701, Generator Loss: 3.680548906326294
1/1 [==============================] - 0s 16ms/step
Epoch: 63, Discriminator Loss: 0.012732304632663817, Generator Loss: 3.7056665420532227
1/1 [==============================] - 0s 15ms/step
Epoch: 64, Discriminator Loss: 0.0130601562559605, Generator Loss: 3.703077554702759
1/1 [==============================] - 0s 16ms/step
Epoch: 65, Discriminator Loss: 0.01594755426049235, Generator Loss: 3.511786699295044
1/1 [==============================] - 0s 16ms/step
Epoch: 66, Discriminator Loss: 0.024549599736928964, Generator Loss: 3.22346568107605
1/1 [==============================] - 0s 18ms/step
Epoch: 67, Discriminator Loss: 0.030175644904375114, Generator Loss: 3.0266990661621094
1/1 [==============================] - 0s 16ms/step
Epoch: 68, Discriminator Loss: 0.024295218288898714, Generator Loss: 3.266944408416748
1/1 [==============================] - 0s 15ms/step
Epoch: 69, Discriminator Loss: 0.016964236274361652, Generator Loss: 3.569794178009033
1/1 [==============================] - 0s 15ms/step
Epoch: 70, Discriminator Loss: 0.012433241121470944, Generator Loss: 3.858548641204834
1/1 [==============================] - 0s 15ms/step
Epoch: 71, Discriminator Loss: 0.009311687201267163, Generator Loss: 4.132821083068848
1/1 [==============================] - 0s 19ms/step
Epoch: 72, Discriminator Loss: 0.007128487341106322, Generator Loss: 4.383180141448975
1/1 [==============================] - 0s 16ms/step
Epoch: 73, Discriminator Loss: 0.00559849292040741, Generator Loss: 4.61116886138916
1/1 [==============================] - 0s 16ms/step
Epoch: 74, Discriminator Loss: 0.00451084552408843, Generator Loss: 4.807563781738281
1/1 [==============================] - 0s 17ms/step
Epoch: 75, Discriminator Loss: 0.00376323121599907, Generator Loss: 4.9800591468811035
1/1 [==============================] - 0s 17ms/step
Epoch: 76, Discriminator Loss: 0.0031905272047244057, Generator Loss: 5.136855125427246
1/1 [==============================] - 0s 19ms/step
Epoch: 77, Discriminator Loss: 0.002742231823504708, Generator Loss: 5.282025337219238
1/1 [==============================] - 0s 16ms/step
Epoch: 78, Discriminator Loss: 0.0023816630710260236, Generator Loss: 5.418517112731934
1/1 [==============================] - 0s 17ms/step
Epoch: 79, Discriminator Loss: 0.0020844512621260875, Generator Loss: 5.547975540161133
1/1 [==============================] - 0s 15ms/step
Epoch: 80, Discriminator Loss: 0.0018375897830647902, Generator Loss: 5.671137809753418
1/1 [==============================] - 0s 20ms/step
Epoch: 81, Discriminator Loss: 0.0016278687551581975, Generator Loss: 5.787439346313477
1/1 [==============================] - 0s 15ms/step
Epoch: 82, Discriminator Loss: 0.0014550281753766825, Generator Loss: 5.896020889282227
1/1 [==============================] - 0s 16ms/step
Epoch: 83, Discriminator Loss: 0.0013100234823788526, Generator Loss: 6.000351905822754
1/1 [==============================] - 0s 15ms/step
Epoch: 84, Discriminator Loss: 0.0011796036537910226, Generator Loss: 6.104654312133789
1/1 [==============================] - 0s 15ms/step
Epoch: 85, Discriminator Loss: 0.0010629953726156438, Generator Loss: 6.2073259353637695
1/1 [==============================] - 0s 16ms/step
Epoch: 86, Discriminator Loss: 0.0009620605898003039, Generator Loss: 6.305461883544922
1/1 [==============================] - 0s 16ms/step
Epoch: 87, Discriminator Loss: 0.0008726277232057097, Generator Loss: 6.399545669555664
1/1 [==============================] - 0s 16ms/step
Epoch: 88, Discriminator Loss: 0.0007956487966556125, Generator Loss: 6.491371154785156
1/1 [==============================] - 0s 20ms/step
Epoch: 89, Discriminator Loss: 0.0007258744029599029, Generator Loss: 6.584486484527588
1/1 [==============================] - 0s 17ms/step
Epoch: 90, Discriminator Loss: 0.0006601120645917113, Generator Loss: 6.678915023803711
1/1 [==============================] - 0s 18ms/step
Epoch: 91, Discriminator Loss: 0.000603695473046173, Generator Loss: 6.770786762237549
1/1 [==============================] - 0s 15ms/step
Epoch: 92, Discriminator Loss: 0.0005510742810215432, Generator Loss: 6.863181114196777
1/1 [==============================] - 0s 21ms/step
Epoch: 93, Discriminator Loss: 0.0005070986317150528, Generator Loss: 6.954822063446045
1/1 [==============================] - 0s 16ms/step
Epoch: 94, Discriminator Loss: 0.0004591155312994033, Generator Loss: 7.041896820068359
1/1 [==============================] - 0s 16ms/step
Epoch: 95, Discriminator Loss: 0.00043754004673246527, Generator Loss: 7.102133274078369
1/1 [==============================] - 0s 18ms/step
Epoch: 96, Discriminator Loss: 0.0003993098499917025, Generator Loss: 7.195601463317871
1/1 [==============================] - 0s 17ms/step
Epoch: 97, Discriminator Loss: 0.00039430787924743527, Generator Loss: 7.181300163269043
1/1 [==============================] - 0s 16ms/step
Epoch: 98, Discriminator Loss: 0.000413235192670669, Generator Loss: 7.1613874435424805
1/1 [==============================] - 0s 15ms/step
Epoch: 99, Discriminator Loss: 0.0007721026631770656, Generator Loss: 6.794639587402344
1/1 [==============================] - 0s 15ms/step
Epoch: 100, Discriminator Loss: 0.0012151042759623032, Generator Loss: 6.107326507568359

In [ ]:
fake_students = []
real_students = []

for epoch, discriminator_loss in enumerate(discriminator_losses, 1):
    if discriminator_loss < threshold:
        fake_students.append((epoch, discriminator_loss))
    else:
        real_students.append((epoch, discriminator_loss))

In [ ]:
print("Fake Students:")
for student in fake_students:
    print("Epoch:", student[0], "Discriminator Loss:", student[1])

Fake Students:
Epoch: 10 Discriminator Loss: 0.07865146896801889
Epoch: 11 Discriminator Loss: 0.06858023791573942
Epoch: 12 Discriminator Loss: 0.059400064405053854
Epoch: 13 Discriminator Loss: 0.052326171193271875
Epoch: 14 Discriminator Loss: 0.0480084412265569
Epoch: 15 Discriminator Loss: 0.04277648194693029
Epoch: 16 Discriminator Loss: 0.04477113636676222
Epoch: 17 Discriminator Loss: 0.04812898358795792
Epoch: 18 Discriminator Loss: 0.048902221489697695
Epoch: 19 Discriminator Loss: 0.05273164715617895
Epoch: 20 Discriminator Loss: 0.05618555401451886
Epoch: 21 Discriminator Loss: 0.06638636766001582
Epoch: 22 Discriminator Loss: 0.07778745144605637
Epoch: 23 Discriminator Loss: 0.08074514009058475
Epoch: 24 Discriminator Loss: 0.08781463885679841
Epoch: 25 Discriminator Loss: 0.08119455527048558
Epoch: 26 Discriminator Loss: 0.07811469794251025
Epoch: 27 Discriminator Loss: 0.0714961844496429
Epoch: 28 Discriminator Loss: 0.07327522244304419
Epoch: 29 Discriminator Loss: 0.07841819641180336
Epoch: 30 Discriminator Loss: 0.09519316325895488
Epoch: 36 Discriminator Loss: 0.09065800912048871
Epoch: 37 Discriminator Loss: 0.058778542443178594
Epoch: 39 Discriminator Loss: 0.0646307051390167
Epoch: 40 Discriminator Loss: 0.07536807667592463
Epoch: 41 Discriminator Loss: 0.07644799351775818
Epoch: 42 Discriminator Loss: 0.06758494675159593
Epoch: 43 Discriminator Loss: 0.0593094453215601
Epoch: 44 Discriminator Loss: 0.06289426982403142
Epoch: 45 Discriminator Loss: 0.08316199481487413
Epoch: 47 Discriminator Loss: 0.09969729185105547
Epoch: 48 Discriminator Loss: 0.06749355793388179
Epoch: 49 Discriminator Loss: 0.0441960617958044
Epoch: 50 Discriminator Loss: 0.03268334404694939
Epoch: 51 Discriminator Loss: 0.02492551893493765
Epoch: 52 Discriminator Loss: 0.019623678457431737
Epoch: 53 Discriminator Loss: 0.01585414593864698
Epoch: 54 Discriminator Loss: 0.013412490574410185
Epoch: 56 Discriminator Loss: 0.02153980638831854
Epoch: 57 Discriminator Loss: 0.013257704578328422
Epoch: 58 Discriminator Loss: 0.0136376563706107
Epoch: 59 Discriminator Loss: 0.01369759067893916
Epoch: 60 Discriminator Loss: 0.013555457815562546
Epoch: 61 Discriminator Loss: 0.01333882659673733
Epoch: 62 Discriminator Loss: 0.0130143715068701
Epoch: 63 Discriminator Loss: 0.012732304632663817
Epoch: 64 Discriminator Loss: 0.0130601562559605
Epoch: 65 Discriminator Loss: 0.01594755426049235
Epoch: 66 Discriminator Loss: 0.024549599736928964
Epoch: 67 Discriminator Loss: 0.030175644904375114
Epoch: 68 Discriminator Loss: 0.024295218288898714
Epoch: 69 Discriminator Loss: 0.016964236274361652
Epoch: 70 Discriminator Loss: 0.012433241121470944
Epoch: 71 Discriminator Loss: 0.009311687201267163
Epoch: 72 Discriminator Loss: 0.007128487341106322
Epoch: 73 Discriminator Loss: 0.00559849292040741
Epoch: 74 Discriminator Loss: 0.00451084552408843
Epoch: 75 Discriminator Loss: 0.00376323121599907
Epoch: 76 Discriminator Loss: 0.0031905272047244057
Epoch: 77 Discriminator Loss: 0.002742231823504708
Epoch: 78 Discriminator Loss: 0.0023816630710260236
Epoch: 79 Discriminator Loss: 0.0020844512621260875
Epoch: 80 Discriminator Loss: 0.0018375897830647902
Epoch: 81 Discriminator Loss: 0.0016278687551581975
Epoch: 82 Discriminator Loss: 0.0014550281753766825
Epoch: 83 Discriminator Loss: 0.0013100234823788526
Epoch: 84 Discriminator Loss: 0.0011796036537910226
Epoch: 85 Discriminator Loss: 0.0010629953726156438
Epoch: 86 Discriminator Loss: 0.0009620605898003039
Epoch: 87 Discriminator Loss: 0.0008726277232057097
Epoch: 88 Discriminator Loss: 0.0007956487966556125
Epoch: 89 Discriminator Loss: 0.0007258744029599029
Epoch: 90 Discriminator Loss: 0.0006601120645917113
Epoch: 91 Discriminator Loss: 0.000603695473046173
Epoch: 92 Discriminator Loss: 0.0005510742810215432
Epoch: 93 Discriminator Loss: 0.0005070986317150528
Epoch: 94 Discriminator Loss: 0.0004591155312994033
Epoch: 95 Discriminator Loss: 0.00043754004673246527
Epoch: 96 Discriminator Loss: 0.0003993098499917025
Epoch: 97 Discriminator Loss: 0.00039430787924743527
Epoch: 98 Discriminator Loss: 0.000413235192670669
Epoch: 99 Discriminator Loss: 0.0007721026631770656
Epoch: 100 Discriminator Loss: 0.0012151042759623032

In [ ]:
print("\nReal Students:")
for student in real_students:
    print("Epoch:", student[0], "Discriminator Loss:", student[1])

Real Students:
Epoch: 1 Discriminator Loss: 0.623881921172142
Epoch: 2 Discriminator Loss: 0.37799350963905454
Epoch: 3 Discriminator Loss: 0.30502695264294744
Epoch: 4 Discriminator Loss: 0.2506387762259692
Epoch: 5 Discriminator Loss: 0.21092747536022216
Epoch: 6 Discriminator Loss: 0.17401077318936586
Epoch: 7 Discriminator Loss: 0.14326616236940026
Epoch: 8 Discriminator Loss: 0.11896649457048625
Epoch: 9 Discriminator Loss: 0.1002053152769804
Epoch: 31 Discriminator Loss: 0.10685841155645903
Epoch: 32 Discriminator Loss: 0.1676519401371479
Epoch: 33 Discriminator Loss: 0.14461377894986072
Epoch: 34 Discriminator Loss: 0.14328525392693336
Epoch: 35 Discriminator Loss: 0.12006446369491641
Epoch: 38 Discriminator Loss: 0.14741291105747223
Epoch: 46 Discriminator Loss: 0.11929800361397531
Epoch: 55 Discriminator Loss: 0.13995973765850067